In [38]:
import os
import sys
import importlib

# Thêm đường dẫn cho các module
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

# Import các module
import import_default
import import_database  
import import_other
import clean_data
import candle_chart

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(clean_data)
importlib.reload(candle_chart)

from import_default import *
from import_database import *
from import_other import *
from clean_data import *
from candle_chart import *

In [39]:
main_stock_list = ['HCM', 'VND', 'CTS', 'BSI']

In [40]:
projection = {
    "_id": 0,
    "date": 1,
    "ticker": 1,
    "open": 1,
    "high": 1,
    "low": 1,
    "close": 1,
    "volume": 1,
    "SMA_20": 1,
    "SMA_60": 1,
    "month_open": 1,
    "month_prev_high": 1,
    "month_prev_low": 1,
    "year_open": 1,
    "year_prev_high": 1,
    "year_prev_low": 1,
    "MPIVOT_P": 1,
    "MFIBO_0382": 1,
    "MFIBO_0618": 1,
    "YPIVOT_P": 1,
	"YFIBO_0382": 1,
    "YFIBO_0618": 1
}

full_stock_ta_dict = {}
for main_stock in main_stock_list:
    today_stock_df = get_mongo_collection(stock_db, "today_stock", find_query={"ticker": main_stock}, projection=projection)
    history_stock_df = get_mongo_collection(stock_db, "history_stock", find_query={"ticker": main_stock}, projection=projection)
    full_stock_ta_df = pd.concat([today_stock_df, history_stock_df], axis=0, ignore_index=True).iloc[:60]
    full_stock_ta_dict[main_stock] = full_stock_ta_df

    full_stock_ta_df.to_csv(f'../output/ta_data/{main_stock}_data.csv', index=False, encoding='utf-8-sig')

In [41]:
line_name_dict = {
    'SMA_20': 'SMA 20',
    'SMA_60': 'SMA 60',
    'month_open': 'MONTH OPEN',
    'month_prev_high': 'LAST MHIGH',
    'month_prev_low': 'LAST MLOW',
    'year_open': 'YEAR OPEN',
    "year_prev_high": 'LAST YHIGH',
    "year_prev_low": 'LAST YLOW',
    "MPIVOT_P": 'MONTH_PIVOT',
    'MFIBO_0382': 'MFIBO 0.382',
    'MFIBO_0618': 'MFIBO 0.618',
    "YPIVOT_P": 'YEAR_PIVOT',
	'YFIBO_0382': 'YFIBO 0.382',
    'YFIBO_0618': 'YFIBO 0.618'
}

chart_config = create_chart_config(
	title_font_size=30,
	axis_font_size=24,
	tag_font_size=24,
	price_tag_font_size=24,
    min_spacing_ratio = 0.05,
	margin=dict(l=20, r=220, t=20, b=20, pad=10)
)

for main_stock in main_stock_list:
    image_name = f'{main_stock}_chart.png'
    
    max_attempts = 5
    for attempt in range(max_attempts):
        try:
            fig, fig_images = create_financial_chart(
                full_stock_ta_dict[main_stock].sort_values('date'), # DataFrame của bạn
                width=1400,
                height=1200,
                line_name_dict=line_name_dict,
                line_columns=list(line_name_dict.keys()),
                chart_config=chart_config,
                symbol_name=main_stock,
                path='../output/ta_chart',
                image_name=image_name
            )
            break
        except Exception as e:
            if attempt == max_attempts - 1:
                raise e
    # fig.show()